In [1]:
from utility_bfvariater import *
import utility_common as common

# creat a BF variater
er = BFVariater()

# 1. Central Value

In [2]:
def saveCentralValues(er):
    print( "save conteral values")

    contralValue = []
    for i in range(4):

        ndata = er.ndata[i]
        nmcbg = er.nmcbg[i].dot(np.ones(3))
        nfake = er.nfake[i]
        a = np.sum(er.a[i] * er.ca, axis=1)

        slv0 = BFSolver3D(a)
        br0 = slv0.solveQuadEqn(slv0.setMeasuredX(nData=ndata, nMcbg=nmcbg+nfake))
        contralValue.append(br0)
    contralValue = np.array(contralValue).reshape(12)

    baseDir = common.getBaseDirectory() 

    print(contralValue)

    np.save(baseDir+'/data/combine/centralValue',contralValue)
saveCentralValues(er)

save conteral values
[0.10943673 0.10941644 0.10394225 0.11246562 0.10990513 0.1035801
 0.1062045  0.10766957 0.1048981  0.1089431  0.10940331 0.10474677]


# 2. uncertainty

### 2.1 Statistical Uncertainty

In [3]:
def saveStatError(er):
    print( "relative statistical uncertainty [%]")

    stat_sigma, stat_covar = [],[]

    for source in ["data","mcbg","mcsg"]:
        sigma, variance_raw = er.errStat(source,returnCovar=True)
        # reshape 4x3x3 to 12x12
        variance = np.zeros([12,12])
        for c in range(4):
            for i in range(3):
                for j in range(3):
                    variance[c*3+i,c*3+j] = variance_raw[c,i,j]
                    
        stat_sigma.append(sigma)
        stat_covar.append(variance)

    stat_sigma = np.array(stat_sigma)  
    stat_covar = np.array(stat_covar) 

    er.io_printErrorForExcelFormat(stat_sigma.transpose(1,0,2))

    s = stat_sigma.shape
    stat_sigma = stat_sigma.reshape(s[0],s[1]*s[2])

    baseDir = common.getBaseDirectory() 
    np.save(baseDir+'/data/combine/stat_sigma',stat_sigma)
    np.save(baseDir+'/data/combine/stat_covar',stat_covar)

saveStatError(er)

relative statistical uncertainty [%]
0.553,0.516,1.217, 0.985,0.867,2.009, 0.769,0.554,1.442, 1.346,0.968,2.421
0.130,0.526,0.575, 0.137,0.498,0.612, 0.622,0.176,0.686, 0.633,0.173,0.681
0.166,0.147,0.397, 0.260,0.222,0.578, 0.223,0.170,0.479, 0.350,0.255,0.673


### 2.2 Systematical Uncertainty

In [4]:
def saveSystError(er):
    print( "relative systematics uncertainty [%]")
    syst_sigma, syst_covar = [],[]

    for source in ["BtmUp","BteUp"]:
        sigma = er.errConstent(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
    for source in ["lumin","mctt","mctw","mcw",'mcz','mcvv',
                   "fakee","fakemu","fakeetau","fakemutau"]:
        sigma = er.errSystem_crossSection(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
        
    for source in ["TopPtReweight","ERecoEff","EIDEff","MuRecoEff","MuIDEff","TauIDEff","JetToTauIDEff",
                   "EPt","MuPt","Tau0Pt","Tau1Pt","Tau10Pt","ETriggerEffProbeSyst","ETriggerEffTagSyst"
                  ]:
        sigma = er.errSystem_downVariation(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
    for source in ["tauReweight1000","tauReweight11000","tauReweight21000",
                   "tauReweight3000","tauReweight13000"
                  ]:
        sigma = er.errSystem_upDownVariation(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
        
    for source in ["Pileup","JES","JER","BTag","Mistag"]:
        sigma = er.errSystem_upDownVariation(source)
        variance = np.outer(sigma.reshape(1,12), sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)      
    
#     for source in ["fsr","isr","ue","meps"]:
#         sigma = er.errSystem_upDownVariationCutResidue(source,cutResidue=3)
#         variance = np.outer(sigma.reshape(1,12), sigma.reshape(1,12))
#         syst_sigma.append(sigma)
#         syst_covar.append(variance) 
    
    syst_sigma = np.array(syst_sigma) 
    syst_covar = np.array(syst_covar) 

    er.io_printErrorForExcelFormat(syst_sigma.transpose(1,0,2))

    s = syst_sigma.shape
    syst_sigma = syst_sigma.reshape(s[0],s[1]*s[2])

    baseDir = common.getBaseDirectory() 
    np.save(baseDir+'/data/combine/syst_sigma',syst_sigma)
    np.save(baseDir+'/data/combine/syst_covar',syst_covar)

saveSystError(er)

relative systematics uncertainty [%]
0.002,0.019,0.030, 0.002,0.019,0.030, 0.002,0.019,0.029, 0.002,0.019,0.029
0.048,0.017,0.101, 0.049,0.017,0.101, 0.032,0.014,0.102, 0.031,0.014,0.102
0.145,0.243,0.031, 0.048,0.069,0.013, 0.101,0.134,0.070, 0.006,0.047,0.010
0.009,0.001,0.109, 0.003,0.012,0.052, 0.018,0.007,0.111, 0.008,0.002,0.004
0.010,0.001,0.114, 0.003,0.013,0.055, 0.018,0.008,0.116, 0.009,0.002,0.004
0.156,0.159,0.142, 0.052,0.051,0.043, 0.152,0.154,0.162, 0.050,0.048,0.073
0.271,1.272,0.148, 0.085,0.369,0.028, 0.692,0.228,0.038, 0.120,0.092,0.104
0.001,0.020,0.011, 0.001,0.008,0.004, 0.017,0.001,0.008, 0.006,0.001,0.001
0.000,0.000,0.000, 0.000,0.000,0.000, 0.374,0.364,0.749, 0.195,0.187,0.329
0.622,0.607,1.231, 0.405,0.384,0.672, 0.000,0.000,0.000, 0.000,0.000,0.000
0.000,0.000,0.000, 0.000,0.000,0.000, 0.194,0.316,1.667, 0.101,0.166,0.898
0.252,0.339,1.724, 0.146,0.193,1.003, 0.000,0.000,0.000, 0.000,0.000,0.000
0.002,0.000,0.023, 0.001,0.003,0.011, 0.004,0.002,0.023, 0.002,

# 3. output Latex Table

In [5]:
%run -i '../scripts/note_uncertainty.py'

In [ ]:
#     for source in ["e","mu","tauID",'jetMisTauID']:
#         sigma = er.errSystem_objectEff(source)
#         variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
#         syst_sigma.append(sigma)
#         syst_covar.append(variance)

In [7]:
sigma = er.errSystem_upDownVariation("fsr")
sigma/0.108*100

array([[-0.7546923 , -0.56336153, -0.78828602],
       [-0.47845538, -0.79648906, -0.80230501],
       [-0.79487902, -0.42558462, -0.6127878 ],
       [-1.07588   , -1.00108001, -1.38707   ]])

In [13]:
sigma = er.errSystem_upDownVariation("isr")
sigma/0.108*100

array([[ 0.61298784,  0.65726433,  1.38221063],
       [ 0.92377239,  0.81140687,  0.91615779],
       [ 0.10179913,  1.07960526,  0.69728675],
       [ 0.13846273,  0.67379168, -0.1033589 ]])

In [15]:
sigma = er.errSystem_upDownVariationCutResidue("isr",3)
sigma/0.108*100

array([[0.47650823, 0.46227043, 0.93861782],
       [0.59470542, 0.33422956, 0.60685315],
       [0.43530199, 0.43752446, 0.90269255],
       [0.06995709, 0.06794519, 0.12052818]])

In [43]:
counts = pd.read_pickle( "../data/counts/count_.pkl")

In [77]:
counts0 = pd.read_pickle( "../data/counts/count_isrDown.pkl")
counts1 = pd.read_pickle( "../data/counts/count_isrUp.pkl")

In [68]:
tr,ch,mc = 0,3,1

acc = common.matrixToArray(counts.acc[tr][ch][mc])
accUp = common.matrixToArray(counts1.acc[tr][ch][mc])
accDown = common.matrixToArray(counts0.acc[tr][ch][mc])

accVar = common.matrixToArray(counts.accVar[tr][ch][mc])
accUpVar = common.matrixToArray(counts1.accVar[tr][ch][mc])
accDownVar = common.matrixToArray(counts0.accVar[tr][ch][mc])

resUp = abs(accUp-acc)/(accVar+accUpVar)**0.5
resDown = abs(accDown-acc)/(accVar+accDownVar)**0.5

name = common.WWBranchNames()
for i in range(21):
    sfUp,sfDown = 0,0
    if resUp[i] > 3:
        sfUp = accUp[i]/acc[i]
        sfDown = accDown[i]/acc[i]
    print("{:20}: {:5.3f}, {:5.3f}".format(name[i],sfUp,sfDown))

$ee$                : 0.000, 0.000
$\mu\mu$            : 0.962, 1.010
$e\mu$              : 0.945, 1.022
$\tau_e\tau_e$      : 0.000, 0.000
$\tau_\mu\tau_\mu$  : 0.000, 0.000
$\tau_e\tau_\mu$    : 0.000, 0.000
$\tau_e\tau_h$      : 0.000, 0.000
$\tau_\mu\tau_h$    : 0.000, 0.000
$\tau_h\tau_h$      : 0.000, 0.000
$e\tau_e$           : 0.000, 0.000
$e\tau_\mu$         : 0.000, 0.000
$e\tau_h$           : 0.000, 0.000
$\mu\tau_e$         : 0.920, 1.040
$\mu\tau_\mu$       : 0.943, 1.023
$\mu\tau_h$         : 0.931, 1.033
$eh$                : 0.528, 1.388
$\mu h$             : 0.916, 1.051
$\tau_eh$           : 0.000, 0.000
$\tau_\mu h$        : 0.920, 1.060
$\tau_hh$           : 0.573, 1.419
$hh$                : 0.502, 1.457


In [88]:
tr = 3
acc = counts.acc[tr].copy()
acc1 = counts1.acc[tr].copy()
acc0 = counts0.acc[tr].copy()

acc01 = (acc1*1/3+acc0*2/3)

accVar = counts.accVar[tr].copy()
acc1Var = counts1.accVar[tr].copy()
acc0Var = counts0.accVar[tr].copy()

res1 = abs(acc1-acc)/(accVar+acc1Var)**0.5
res0 = abs(acc0-acc)/(accVar+acc0Var)**0.5
res01 = abs(acc0-acc1)/(acc1Var+acc0Var)**0.5

for ch in range(4):
    mc=1
    for i in range(6):
        for j in range(6):
            if res01[ch,mc][i,j]<3:
                acc1[ch,mc][i,j] = acc01[ch,mc][i,j]
                acc0[ch,mc][i,j] = acc01[ch,mc][i,j]

In [89]:
# up tuning
slv1 = BFSolver3D( np.sum(acc1*er.ca,axis=1) )
BW1  = slv1.solveQuadEqn(slv1.setMeasuredX(nData=counts1.ndata[tr], nMcbg=counts1.nmcbg[tr].dot(np.ones(3))+counts1.nfake[tr]))
# down tuning
slv2 = BFSolver3D( np.sum(acc0*er.ca,axis=1) )
BW2  = slv2.solveQuadEqn(slv2.setMeasuredX(nData=counts0.ndata[tr], nMcbg=counts0.nmcbg[tr].dot(np.ones(3))+counts0.nfake[tr]))
# differentce between up and down tuning
print((BW1-BW2)/2/0.1080*100)

[0.0988434  0.09565056 0.17098258]


In [85]:
print(counts.ndata[0]-counts0.ndata[0])
print(counts.nmcbg[0]-counts0.nmcbg[0])
print(counts.nfake[0]-counts0.nfake[0])

[0. 0. 0. 0.]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[0. 0. 0. 0.]


In [24]:
counts1.accVar[0][3][1]**0.5/counts1.acc[0][3][1]

array([[1.23812276e+00, 5.44267664e-03,            inf, 2.26110848e-02,
        6.02663712e-01, 1.20661118e-01],
       [5.44267664e-03, 7.80294975e-03, 1.27426556e-02, 1.38241656e-02,
        5.41245226e-03, 1.07055726e-03],
       [           inf, 1.27426556e-02,            inf, 5.33164419e-02,
                   inf, 2.40599071e-01],
       [2.26110848e-02, 1.38241656e-02, 5.33164419e-02, 6.28052574e-02,
        2.26312636e-02, 4.59895401e-03],
       [6.02663712e-01, 5.41245226e-03,            inf, 2.26312636e-02,
        4.35954570e-01, 1.03693105e-01],
       [1.20661118e-01, 1.07055726e-03, 2.40599071e-01, 4.59895401e-03,
        1.03693105e-01, 3.80247767e-02]])

In [2]:
sigma, variance_raw = er.errStat("mcsg",returnCovar=True)

In [4]:
sigma/0.108*100

array([[0.53401089, 0.45867545, 1.15690868],
       [0.95222688, 0.80551395, 1.82853771],
       [0.64549883, 0.52774019, 1.38175311],
       [1.2019165 , 0.93939064, 2.2449766 ]])